In [1]:
import numpy as np
import pandas
import itertools
import inspect
import uuid

uuid.uuid3(uuid.NAMESPACE_DNS, 'python.org')

Matplotlib is building the font cache using fc-list. This may take a moment.


UUID('6fa459ea-ee8a-3ca4-894e-db77e160355e')

In [0]:
# Matrice de MP
# Maille MP :
#   - indice temps, indice étape, indice produit, indice mp
# Maille Produit :
#   - indice temps, indice étape, indice produit
# Maille Compagnie :
#   - indice temps, indice étape
# Maille Top :
#   - None / indice temps / indice temps, indice étape

CUSTOM = 0 # -> pour définir des étapes de calcul (bop, eop, ...)
YEARLY = 1
QUARTERLY = 3
BIANNUALY = 6
MONTHLY = 12
DAILY = 365

def md5_id(name):
    return uuid.uuid3(uuid.NAMESPACE_DNS, name).hex

class Mesh:
    None
    # Etend la classe array
    # initialisation à partir de Meshs et de cash flows et d'une liste de variables et d'indices
    # A chaque variable de la liste est associé une méthode qui définie la variable (-> cette méthode est récupérée par le builder)
    # A chaque sous-Mesh est associé un getter
    # Une méthode build renvoie le graph de dépendance de la mesh
    # Une méthode exec exécute les calculs sur des inputs du graph de dépendance
    #    - la méthode exec doit pouvoir gérer le parallélisme et la gestion de la mémoire (éventuellement écriture sur le disque)
    # 
    
class Placeholder:
    id_ = None
    name = None
    def __init__(self, name):
        self.id_ = md5_id(name)
        self.name = name
    
class Constant(Placeholder):
    value = None
    def __init__(self, name, value):
        super().__init__(name)
        self.value = value
    def __eq__(self, other):
        return self.id_ == other.id_


class TimeStep(Placeholder):
    """Todo: add multiply method """
    def __init__(self, shift = 0, name = "main", ref = Constant("TimeStep_ref", 0), base = YEARLY):
        super().__init__(name)
        self.shift = shift
        self.ref = ref
        self.base = YEARLY # YEARLY, MONTHLY, DAILY
        self.values = {}
        
    def __add__(self, value):
        if isinstance(value, TimeStep):
            raise ArithmeticError("Cannot use +, -, * or / operators with TimeSteps objects. Please use set_ref to set a relative TimeStep or monthly(), quarterly(), biannualy(), yearly() to set the base to use.")
        else:
            return TimeStep(shift = self.shift + value, name = self.name)
        
    def __sub__(self, other):
        if isinstance(other, TimeStep):
            raise ArithmeticError("Cannot use +, -, * or / operators with TimeSteps objects. Please use set_ref to set a relative TimeStep or monthly(), quarterly(), biannualy(), yearly() to set the base to use.")
        else:
            return TimeStep(shift = self.shift - other, name = self.name)
    
    def set_ref(self, other):
        if isinstance(other, TimeStep):
            TimeStep(shift = self.shift - value, name = self.name, ref = other)
    
    def yearly():
        self.base = YEARLY
        
    def monthly():
        self.base = MONTHLY
    
    def daily():
        self.base = DAILY
        
    def evaluate(self, value):
        if self.values.get(value, False):
            
        if value == self.ref.value:
            return self.ref.value
        elif value > self.ref.value:
            return 1 + self.evaluate(value - 1)
        elif value < self.ref.value:
            return -1 + self.evaluate(value + 1)
        
    def formula(self, value):
        if value == self.ref.value:
            return self.ref.value
        elif value > self.ref.value:
            return 1 + self.formula(value - 1)
        elif value < self.ref.value:
            return -1 + self.formula(value + 1)
        
        if self.shift == 0:
            return self.ref.value
        else:
            return self.__sub__(1)
        
        
    
class Hypothesis(Placeholder):
    """Different categories : numeric, factor, string (in case of id)"""
    category = None
    def __init__(self, name):
        super().__init__(name)
        self.shift = shift

    
class Flow(Placeholder):
    """Time dependent placeholder"""
    size = None
    # dim = None
    # dimnames = None
    dependencies = {"in":[], "out":[]}
    built = False
    values = []
    computed = []
    def __init__(self, size = 1, name = "CF_" + str(random.randint(1, 100))):
        super().__init__(name)
        self.size = size
    def __add__(self, cf):
        #[self.cf_id, self.]
        self.dependencies["in"].append(self.cf_id)
        cf.dependencies["in"].append(cf.cf_id)
    def __getitem__(self, key):
        return([self.id_, key])
    def __setitem__(self, ):
        None

In [0]:
t = TimeStep(shift = 3)
#t = t - 2
#Placeholder("itt_rate")
#print(t.shift)
#print(t.id_)
#(t - t).shift
print(t.shift)
t = t + 1
print(t.shift)
t.evaluate(-15)

3
4


-15

In [0]:
0 == False

True

In [0]:
my_cf = CashFlow(10, "my_cf")
my_cf[2]

['25e14044f12c30a89af29abb5778eef2', 2]

In [0]:
a = np.arange(1, 10)
a[11]

IndexError: index 11 is out of bounds for axis 0 with size 9

In [0]:
import itertools
a = (range(i) for i in range(4, 6))
for i in itertools.product(*a):
  print(i)

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)


In [0]:
a = np.ones((10, 2))
a[1, -2]


1.0

In [51]:

class Flow:
    def __init__(self, model, mesh, name, shape):
        self.shape = shape # 1 Dimension for now
        self.model = model
        self.mesh = mesh
        self.computed = np.zeros(shape, dtype = bool)
        self.values = np.zeros(shape, dtype = float) * np.nan # not very clean
        self.compute_mode = False
        
    def __getitem__(self, *args):
        if self.compute_mode & (not self.computed[args[0]]):
            self.formula(args[0])
        return self.values.__getitem__(args[0])
    
    def __setitem__(self, *args):
        self.values[args[0]] = args[1]
        self.computed[args[0]] = True
    
    def slide_window(self):
        pass
    
    def compute(self, index):
        """
        Args::
        - index : index of the value to compute. Compute all values if index == -2.
                  Compute the last value if index == -1

        if index == -1:
        return self[self.shape - 1]
        elif index == -2:
        ranges = (range(i) for i in self.shape)
        for index in itertools.product(*ranges):
          self[index]
        return self[self.shape - 1]
        """
        self.compute_mode = True
        return self[index]
  
    def formula(self, index):
        s = self
        m = self.mesh
        top = self.model
        
        # Formula stuff...
        self[index] = None # some stuff...
    def resize(self, shape):
        self.computed = np.zeros(shape, dtype = bool)
        self.values = np.zeros(shape, dtype = float) * np.nan

class Mesh:
    def __init__(self, model, name, shape, shape_names):
        self.shape = shape
        self.model = model
        self.shape_names = shape_names
        
        self.flow = {}
        self.mesh = {}
        
    def compute(self, index):
        pass
        
    def __getitem__(self, item):
        return self.flow[item]
    
    def resize(self, shape):
        for key in self.flow.keys():
            self.flow[key].resize(shape)

        


class Model:
    mesh = {}
    report = {}
    def __init__(self):
        pass
    def __getitem__(self, item):
        return self.mesh[item]
        

In [52]:
{"a":0}.keys()

dict_keys(['a'])

In [53]:

        
class FlowTest(Flow):
    def __init__(self, model, mesh, name, shape):
        super().__init__(model, mesh, name, shape)
        self.T1, self.T2 = self.shape[0], self.shape[1]
        
    def formula(self, index):
        t1, t2, mp = index
        T1, T2 = self.T1, self.T2
        if (t1 == 0) & (t2 == 0):
            self[t1, t2, mp] = 0
        elif (t2 == 0):
            self[t1, t2, mp] = self[t1-1, T2-1, mp] + 1
        else:
            self[t1, t2, mp] = self[t1, t2-1, mp] + 1
    

flow = FlowTest(None, None, "FlowTest", (10, 10, 3))
print(flow.compute_mode)
print(flow.compute_mode)
flow.compute((5, 3, 1))
flow.values[:,: , 2]

False
False


array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [54]:
class Report:
    def __init__(self, model, name):
        self.model = model
        self.name = name
    def export(self):
        pass
    def compute(self):
        pass


class Report_central(Report):
    def __init__(self, model, name):
        super().__init__(self, model, name)
    def compute(self):
        model["prod_tra"].compute(-1)

In [55]:
a = np.ones((3, 4))
print(a)
a.resize(5, 6)
print(a)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [56]:
class Credit(Mesh):
    def __init__(self, model, name, shape, shape_names):
        super().__init__(model, name, shape, shape_names)
        
        self.flow["flow_test"] = FlowTest(model, self, "flow_test", shape)
        #self.mesh["submesh1"] = Mesh(model, "submesh1", [10, 2, 1], ["time", "etape", "credit", "submesh1"])

        
class Prod_tra(Mesh):
    def __init__(self, *args):
        super().__init__(*args)
        
class Goc_insu(Mesh):
    def __init__(self, *args):
        super().__init__(*args)
        
class Goc_reass(Mesh):
    def __init__(self, *args):
        super().__init__(*args)


class Pv_prod_tra(Mesh):
    def __init__(self, *args):
        super().__init__(*args)
        
    
class PVIF(Model):
    T = 10
    substeps = 2
    def __init__(self):
        super().__init__()
        self.mesh["credit"] = Credit(self, "credit", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["prod_tra"] = Prod_tra(self, "prod_tra", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["goc_insu"] = Goc_insu(self, "goc_insu", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["goc_reass"] = Goc_reass(self, "goc_reass", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.mesh["pv_prod_tra"] = Pv_prod_tra(self, "pv_prod_tra", [self.T, self.substeps, 1], ["time", "etape", "mp"])
        self.report["central"] = Report(self, "central")
        self.data = {}

    def on_data_fed(self):
        """
        Resize data and then additionnal operations
        """
        self["credit"].resize((self.T, self.substeps, len(self.data["mp"]) ))

    def feed_data_central(self):
        """
        Just assign values to self.data
        """
        self.data["mp"] = pandas.DataFrame({"id": list(range(10)), 
                                          "csr": [100] * 10, 
                                          "prod_activated": [True] * 3 + [False] * 7})
        self.on_data_fed()

    def fetch_report_central(self):
        self.report["central"].compute()
        self.report["central"].export()
    

pvif = PVIF()
pvif.feed_data_central()

print(pvif["credit"]["flow_test"][2, 1, 0])
pvif["credit"]["flow_test"][2, 1, 0] = 10
print(pvif["credit"]["flow_test"][2, 1, 0])

print(pvif["credit"].model["credit"])
print(pvif["credit"])

nan
10.0


In [63]:
pvif["credit"]["flow_test"]

In [60]:
pvif.mesh["credit"].flow["flow_test"].values

array([[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [10., nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, 

In [31]:
[True]*3+[False]*7

[True, True, True, False, False, False, False, False, False, False]

In [61]:
pvif["credit"]["flow_test"].values

array([[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [10., nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[nan, nan, nan, 

In [0]:
pvif["credit"]["flow_test"].compute((5, 1, 1))

IndexError: ignored

In [0]:
pvif["credit"]["flow_test"].values

array([[[nan],
        [nan]],

       [[nan],
        [nan]],

       [[nan],
        [10.]],

       [[11.],
        [12.]],

       [[13.],
        [14.]],

       [[15.],
        [16.]],

       [[nan],
        [nan]],

       [[nan],
        [nan]],

       [[nan],
        [nan]],

       [[nan],
        [nan]]])

In [0]:
pvif["credit"]["flow_test"][4, 1]

14.0

In [0]:
pvif["credit"]["flow_test"].compute(-2)

TypeError: ignored

In [0]:
class Produit(Mesh):
    cashflows = {}
    def __init__(self):
        super().__init()__
    def get_cash_flow(self, name):
        return self.cashflows[name]
    def set_cash_flow(self, name, fun):
        self.cashflows[name].set(fun)